### This file is dedicated to scrape and download data from domain.com.au

Yuecheng Wang Aug 30, modify from ./scripts/scrape.py

In [1]:
import os
import re
from tqdm import tqdm
from collections import defaultdict
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import pandas as pd
import requests
import time

In [2]:
# Constants
BASE_URL = "https://www.domain.com.au"
property_metadata = []

In [3]:
# Adjust the postcode ranges as needed
postcodes = list(range(3000, 4000)) + list(range(8000, 9000))


for postcode in postcodes:
    N_PAGES = range(1, 3)
    url_links = []

    for page in N_PAGES:
        url = BASE_URL + f"/rent/?postcode={postcode}&sort=dateupdated-desc&page={page}"
        print(f"Visiting {url}")
        try:
            bs_object = BeautifulSoup(urlopen(Request(url, headers={'User-Agent': "PostmanRuntime/7.6.0"})), "html.parser")
        except Exception as e:
            print(f"Failed to load page {url}: {e}")
            continue

        result_section = bs_object.find("ul", {"data-testid": "results"})
        if not result_section:
            print(f"No results found on page {url}")
            continue

        index_links = result_section.findAll("a", href=re.compile(f"{BASE_URL}/*"))
        for link in index_links:
            if link.get('class') and 'address' in link['class']:
                url_links.append(link['href'])

    pbar = tqdm(url_links)
    success_count, total_count = 0, 0
    for property_url in pbar:
        total_count += 1
        try:
            bs_object = BeautifulSoup(urlopen(Request(property_url, headers={'User-Agent': "PostmanRuntime/7.6.0"})), "html.parser")
            time.sleep(0.5)  # Delay to avoid rate limiting
        except Exception as e:
            print(f"Failed to load property page {property_url}: {e}")
            continue

        try:
            # Extract property details
            address = bs_object.find("h1", {"class": "css-164r41r"}).text if bs_object.find("h1", {"class": "css-164r41r"}) else "N/A"
            cost_text = bs_object.find("div", {"data-testid": "listing-details__summary-title"}).text if bs_object.find("div", {"data-testid": "listing-details__summary-title"}) else "N/A"

            # Initialize room details
            bedrooms, bathrooms = "N/A", "N/A"
            rooms_section = bs_object.find("div", {"data-testid": "property-features"})
            if rooms_section:
                rooms = rooms_section.findAll("span", {"data-testid": "property-features-text-container"})
                # Extract bedroom and bathroom details separately
                for feature in rooms:
                    text = feature.text.lower()
                    if 'bed' in text:
                        bedrooms = re.findall(r'\d+', text)[0] if re.findall(r'\d+', text) else "N/A"
                    elif 'bath' in text:
                        bathrooms = re.findall(r'\d+', text)[0] if re.findall(r'\d+', text) else "N/A"



            # Attempt to find the link that contains the coordinates
            try:
                href = bs_object.find("a", {"target": "_blank", 'rel': "noopener noreferrer"})
                href = href.attrs['href'] if href else ""
                # Extract coordinates from the URL using regex
                coordinates = [float(coord) for coord in re.findall(r'destination=([-\s,\d\.]+)', href)[0].split(',')] if href else [None, None]
            except (AttributeError, IndexError, ValueError):
                coordinates = [None, None]

            # Extract closest government secondary school information
            gov_school_name, gov_school_distance = "N/A", "N/A"
            school_section = bs_object.find("div", {"data-testid": "listing-details__school-catchment"})
            if school_section:
                gov_section = school_section.find("h3", string="Government School Catchment")
                if gov_section:
                    gov_secondary = gov_section.find_next("ul")
                    if gov_secondary:
                        gov_schools = gov_secondary.find_all("li", {"data-testid": "fe-co-school-catchment-school"})
                        for school in gov_schools:
                            # Check if the school tag includes secondary levels like 7-12, 7-10, or 10-12
                            levels = school.find_all("li", {"data-testid": "fe-co-school-catchment-tags"})
                            if any('7-12' in level.text or '7-10' in level.text or '10-12' in level.text for level in levels):
                                gov_school_name = school.find("h4").text
                                gov_school_distance = school.find("p").text
                                break  # Stop after finding the first secondary school

            # Extract Neighbourhood Insights age data
            age_under_20, age_20_39, age_40_59, age_60_plus = "N/A", "N/A", "N/A", "N/A"
            neighbourhood_section = bs_object.find("div", {"data-testid": "listing-details-neighbourhood-insights-container"})
            if neighbourhood_section:
                age_table = neighbourhood_section.find("table", {"class": "css-8atqhb"})
                if age_table:
                    age_rows = age_table.find_all("tr", {"data-testid": "neighbourhood-insights__age-brackets-row"})
                    for row in age_rows:
                        age_label = row.find("td", {"data-testid": "neighbourhood-insights__label"}).text
                        age_value = row.find("div", {"data-testid": "bar-value"}).text
                        if "under 20" in age_label.lower():
                            age_under_20 = age_value
                        elif "20 - 39" in age_label.lower():
                            age_20_39 = age_value
                        elif "40 - 59" in age_label.lower():
                            age_40_59 = age_value
                        elif "60+" in age_label.lower():
                            age_60_plus = age_value

            # Append the flattened property data to the list
            property_metadata.append({
                'Address': address,
                'Cost': cost_text,
                'Bedrooms': bedrooms,
                'Bathrooms': bathrooms,
                'Coordinates': coordinates,
                'Closest Gov Secondary School': gov_school_name,
                'Gov Secondary Distance': gov_school_distance,
                'Age under 20': age_under_20,   # New feature for age data
                'Age 20-39': age_20_39,         # New feature for age data
                'Age 40-59': age_40_59,         # New feature for age data
                'Age 60+': age_60_plus,         # New feature for age data
                'Postcode': postcode
            })
            success_count += 1

        except AttributeError as e:
            print(f"Issue with {property_url}: {e}")

        if total_count > 0:
            pbar.set_description(f"{(success_count / total_count * 100):.0f}% successful")

# Save to CSV
output_relative_dir = '../../data/raw/domain/'
if not os.path.exists(output_relative_dir):
    os.makedirs(output_relative_dir)

csv_file_path = f'{output_relative_dir}all_postcodes.csv'
df = pd.DataFrame(property_metadata)
df.to_csv(csv_file_path, index=False)
print(f"Data saved to {csv_file_path}")


Visiting https://www.domain.com.au/rent/?postcode=3000&sort=dateupdated-desc&page=1
Visiting https://www.domain.com.au/rent/?postcode=3000&sort=dateupdated-desc&page=2


100% successful: 100%|██████████████████████████████████████████████████████████████████| 40/40 [00:48<00:00,  1.21s/it]


Visiting https://www.domain.com.au/rent/?postcode=3001&sort=dateupdated-desc&page=1
No results found on page https://www.domain.com.au/rent/?postcode=3001&sort=dateupdated-desc&page=1
Visiting https://www.domain.com.au/rent/?postcode=3001&sort=dateupdated-desc&page=2
No results found on page https://www.domain.com.au/rent/?postcode=3001&sort=dateupdated-desc&page=2


0it [00:00, ?it/s]

Visiting https://www.domain.com.au/rent/?postcode=3002&sort=dateupdated-desc&page=1


...

0it [00:00, ?it/s]

Visiting https://www.domain.com.au/rent/?postcode=8999&sort=dateupdated-desc&page=1


No results found on page https://www.domain.com.au/rent/?postcode=8999&sort=dateupdated-desc&page=1
Visiting https://www.domain.com.au/rent/?postcode=8999&sort=dateupdated-desc&page=2
No results found on page https://www.domain.com.au/rent/?postcode=8999&sort=dateupdated-desc&page=2


0it [00:00, ?it/s]

Data saved to ../../data/raw/domain/all_postcodes.csv


In [4]:
# download past data
url = "https://www.dffh.vic.gov.au/moving-annual-rents-suburb-march-quarter-2023-excel"

output_directory = "../../data/raw/domain/"
file_name = "past_data.xlsx"
output_path = os.path.join(output_directory, file_name)

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Write the content of the response to the specified path
    with open(output_path, 'wb') as file:
        file.write(response.content)
    print(f"File downloaded successfully and saved as {output_path}")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

File downloaded successfully and saved as ../../data/raw/domain/past_data.xlsx
